In [3]:
import sys
import os
sys.path.insert(0, os.path.abspath(os.path.join('..', '..')))
from src import *
from itertools import product

import numpy as np
import pandas as pd
import scipy.optimize as opt

from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

In [4]:
moments = np.array([-2, -1, 2, 3, 4])
ps = np.array([0.85, 0.9, 0.95, 1.0, 1.05, 1.1, 1.15])
system_sizes = np.array([100, 200, 400, 800, 1600, 3200])#, 6400]
realizations = 10

multifractal_data = np.load("../../data/multifractal_data.npy")

In [5]:
# this is an (Nx5) matrix where each row has the following structure
# row 1: IPR_avg_1 | IPR_err_1 | L_1 | p_1 | lamb_1
# row 2: IPR_avg_2 | IPR_err_2 | L_2 | p_2 | lamb_2
# and so on...

ipr_data = np.load("../../data/AVG_IPR_joined_data.npy")
ipr_df = pd.DataFrame(ipr_data, columns = ['average', 'error', 'L', 'p', 'lambda'])
ipr_ps = np.sort(pd.unique(ipr_df['p']))
ipr_Ls = np.array(np.sort(pd.unique(ipr_df['L'])), dtype = int)


In [6]:
def f(x, a, c2): return c2*(x**a)

powers = []
gradients = []
for p in ipr_ps:
    curve_data = ipr_df[ipr_df['p'] == p]
    res = opt.curve_fit(
        f, curve_data['L'], 1/curve_data['average'], 
        bounds=([0, -np.inf], [np.inf, np.inf])
    )
    powers.append(res[0][0])
    gradients.append(res[0][1])

In [78]:
%matplotlib qt

plot_moment = [0, 1, 1, 1, 0]
plot_p = [0, 0, 1, 1, 1, 1, 0]

# Plotting parameters
scaling = 2.5
title_font_size = 10*scaling
plt.rcParams.update({
    "font.family": "Times New Roman",
    "font.size": 10*scaling,
    #"text.usetex": True,
})

# Figure setup
pt = 0.0138889
fig, ax = plt.subplots(
    nrows=4, figsize=(246*pt*scaling, 250*pt*scaling),
    gridspec_kw={
        'left': 0.12, 'right': 0.85, 'top': 0.98, 'bottom': 0.055, 'hspace': 0.65,
        'height_ratios': (0.3, 0.15, 0.15, 0.15)
    }
)

# IPR data
for L in ipr_Ls:
    if L != 1000:
        ax[0].plot(
            ipr_ps, 
            1/np.array(ipr_df[ipr_df['L'] == L].sort_values('p')['average']),
            ls = '--'
        )
for L in ipr_Ls:
    if L != 1000:
        ax[0].scatter(
            ipr_ps,
            1/np.array(ipr_df[ipr_df['L'] == L].sort_values('p')['average']),
            marker='o',
            label=r'{}'.format(L)
        )
ax[0].set_xlabel(r"$p$", labelpad=-9*scaling)
ax[0].set_ylabel(r"$\mathrm{PR}$")
ax[0].set_xticks((0, 2, 4, 6, 8, 10))
ax[0].set_yscale('log')
ax[0].legend(
    loc=7, 
    bbox_to_anchor=(0.0, 0.0, 1.22, 1.0), 
    borderpad=0.1, 
    handletextpad=0.0,
    handlelength=1.0
)

# IPR inset
inset_ax = inset_axes(ax[0], width="50%", height="50%", loc=4,
                   bbox_to_anchor=(.0, .2, 0.9, 1.0), bbox_transform=ax[0].transAxes)
inset_ax.set_xlabel(r"$p$", labelpad=-9*scaling)
inset_ax.set_title(r"$\mathrm{PR}(p)=p^a/c$", fontsize=title_font_size)
inset_ax.set_xticks((0, 3, 7, 10))

color = (0.05, 0.4, 0.6)
inset_ax.plot(ipr_ps, np.abs(powers), color=color)
inset_ax.set_ylabel(r"$a$", color=color)
inset_ax.tick_params(axis='y', labelcolor=color)

color = (1.0, 0.5, 0.2)
inset_ax_twin = inset_ax.twinx()
inset_ax_twin.plot(ipr_ps, 1/np.abs(gradients), color=color)
inset_ax_twin.set_ylabel(r"$c$", color=color)
inset_ax_twin.tick_params(axis='y', labelcolor=color)

# Multifractality data
for (k, q) in enumerate(moments):
    if plot_moment[k]:
        for (i, p) in enumerate(ps):
            if plot_p[i]:
                ax[k].scatter(
                    system_sizes, 
                    multifractal_data[i, :, k],
                    linewidth=2.0, label="{}".format(p)
                )
                ax[k].plot(
                    system_sizes, 
                    multifractal_data[i, :, k], 
                    linewidth=0.5
                )
        ax[k].set_xlabel(r"$L$", labelpad=-9*scaling)
        ax[k].set_ylabel(r"$D_q$")
        ax[k].set_xscale('log')
        ax[k].set_title(r"$q={}$".format(q), fontsize=title_font_size)
ax[2].legend(
    loc=7, 
    bbox_to_anchor=(0.0, 0.0, 1.215, 1.0),
    borderpad=0.1,
    handletextpad=0.0,
    handlelength=1.0
)
plt.show()


In [79]:
plt.savefig("../../paper_figs/fractal_dimensions.png")
